In [1]:
import sys, os
sys.path.insert(1, os.path.abspath(os.path.join(os.getcwd(), '../')))
from stochss_compute.cloud.ec2 import EC2Cluster, p

In [2]:
cluster = EC2Cluster()

In [3]:
cluster.launch_single_node_cluster()

StochSS-Compute root key detected in working directory. Using "./sssc-root.pem".
vpcId: vpc-0aaae1dae8fe0eac7
subnetId: subnet-002920bdc4dd9956a
sgId: sg-06714cc19d6fbb218
Instance "i-08836d84c9c484e16" is now ready.
head: <stochss_compute.cloud.ec2.EC2Cluster.Instance object at 0x7f1b3b7b61c0>


In [6]:
vpcId = 'vpc-0aaae1dae8fe0eac7'
vpc = cluster.resources.Vpc(vpcId)

In [19]:
for igw in vpc.internet_gateways.all():
    print(igw)
    igw.detach_from_vpc(VpcId=vpc.vpc_id)
    igw.delete()

ec2.InternetGateway(id='igw-0237ad6e9ae33d956')


In [20]:
for rtb in vpc.route_tables.all():
    print(rtb)

ec2.RouteTable(id='rtb-044ff876b7ed00afd')
ec2.RouteTable(id='rtb-0a74215f3513cab5f')


In [4]:
head = cluster.instances[0]

In [3]:
# cluster.create_root_key()
vpcId = cluster.create_default_vpc()
print(f'vpcId: {vpcId}')


vpcId: vpc-07a07397c4d088fec


In [4]:
subnetId = cluster.create_default_subnet(vpcId)
print(f'subnetId: {subnetId}')


subnetId: subnet-0e6cd9103ec216668


In [5]:
subnet = cluster.resources.Subnet(subnetId)

In [6]:
subnet.map_public_ip_on_launch

True

In [7]:
sgId = cluster.create_default_security_group(vpcId)
print(f'sgId: {sgId}')


sgId: sg-05c171ba5ca329ca2


In [4]:
head = cluster.launch_instances(subnetId="", securityGroupId="")
print(f'head: {head}')


Instance "i-0e10ef4e68257045a" is now ready.
head: <stochss_compute.cloud.ec2.EC2Cluster.Instance object at 0x7f646e1c5640>


In [27]:
head.authorize_SSH()

ClientError: An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: 0.0.0.0/0, TCP, from port: 22, to port: 22, ALLOW" already exists

In [5]:
igw_response = cluster.client.create_internet_gateway()

In [6]:
igwId = igw_response['InternetGateway']['InternetGatewayId']
vpc = cluster.resources.Vpc('vpc-07a07397c4d088fec')

In [7]:
igw_response

{'InternetGateway': {'Attachments': [],
  'InternetGatewayId': 'igw-0dfe61b897cb35316',
  'OwnerId': '946980283210',
  'Tags': []},
 'ResponseMetadata': {'RequestId': 'fdcd67a5-9f80-44e5-b461-d30b3d2e8743',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fdcd67a5-9f80-44e5-b461-d30b3d2e8743',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '437',
   'date': 'Mon, 20 Jun 2022 17:01:55 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [8]:
vpc.attach_internet_gateway(InternetGatewayId=igwId)

ClientError: An error occurred (InvalidParameterValue) when calling the AttachInternetGateway operation: Network vpc-07a07397c4d088fec already has an internet gateway attached

In [15]:
vpc.create_route_table()

ec2.RouteTable(id='rtb-09714e8cfb9e31a6c')

In [16]:
rtbId = 'rtb-09714e8cfb9e31a6c'

In [9]:
cluster.client.create_route(RouteTableId="rtb-0674cb67d39179144",GatewayId="igw-09222916c2f305eca",DestinationCidrBlock='0.0.0.0/0')

{'Return': True,
 'ResponseMetadata': {'RequestId': 'a15a0d3d-bd71-4a6c-ab86-f76c4cdbdd7a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a15a0d3d-bd71-4a6c-ab86-f76c4cdbdd7a',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '223',
   'date': 'Mon, 20 Jun 2022 17:05:15 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [23]:
cluster.terminate_all_instances()

['i-05d7ce43c3305c333']
